In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/kaggle/input/used-car-dataset-ford-and-mercedes/audi.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

No null values, perfect

In [ ]:
df['model'].value_counts()

In [ ]:
df['transmission'].value_counts()

In [ ]:
df['fuelType'].value_counts()

In [ ]:
cat_cols = ['model', 'transmission', 'fuelType']

Time to preproces. I'm gonna use Scikit-Learns One-Hot Encoding

In [ ]:
from sklearn.preprocessing import OneHotEncoder

OHE = OneHotEncoder(handle_unknown = 'ignore', sparse = False)

df_OHE = pd.DataFrame(OHE.fit_transform(df[cat_cols]))

df_OHE.index = df.index

df_num = df.drop(cat_cols, axis=1)

df_OHE_concat = pd.concat([df_num, df_OHE], axis=1)

In [ ]:
df_OHE_concat.head()

In [ ]:
print(df.shape)
print(df_OHE.shape)
print(df_OHE_concat.shape)

In [ ]:
X = df_OHE_concat.drop('price', axis=1)
y = df['price']

Split Dataset

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3)

In [ ]:
print(X_train.shape)
print(X_val.shape)
print(y_train.shape)
print(y_val.shape)

Time to evaluate Decision Tree, Random Forest, Linear Regression and KNeighbors. I'm gonna use MAE for calculating accuracy

In [ ]:
from sklearn.metrics import mean_absolute_error as mae

In [ ]:
from sklearn.tree import DecisionTreeRegressor
dt = DecisionTreeRegressor(max_leaf_nodes=900, random_state=1)

dt.fit(X_train, y_train)
dt_pred = dt.predict(X_val)
dt_mae = mae(dt_pred, y_val)
print('MAE of Decision Tree is: {:,.0f}'.format(dt_mae))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state=1)

rf.fit(X_train, y_train)
rf_pred = dt.predict(X_val)
rf_mae = mae(rf_pred, y_val)
print('MAE of Random Forest is: {:,.0f}'.format(rf_mae))

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

lr.fit(X_train, y_train)
lr_pred = lr.predict(X_val)
lr_mae = mae(lr_pred, y_val)
print('MAE of Linear Regression is: {:,.0f}'.format(lr_mae))

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV

knn = KNeighborsRegressor()
params = {'n_neighbors':[2,3,4,5,6,7,8,9]}

model = GridSearchCV(knn, params, cv=5)
model.fit(X_train, y_train)
model.best_params_

In [ ]:
kn = KNeighborsRegressor(n_neighbors=4)

kn.fit(X_train, y_train)
kn_pred = kn.predict(X_val)
kn_mae = mae(kn_pred, y_val)
print('MAE of KNeighbors is: {:,.0f}'.format(kn_mae))

In [ ]:
print(dt.score(X_val, y_val)*100)
print(rf.score(X_val, y_val)*100)
print(lr.score(X_val, y_val)*100)
print(kn.score(X_val, y_val)*100)